In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle

with open("cifar-10-batches-py/data_batch_1", "rb") as f:
    data = cPickle.load(f)
    
    

In [ ]:
color_X = data['data'].reshape(-1, 3, 32, 32)
color_X = np.rollaxis(color_X, 1, 4)

print color_X.shape

In [ ]:
grayscale_weights = np.array([0.3, 0.59, 0.11])
gray_X = np.dot(color_X, grayscale_weights)

print gray_X.shape

In [ ]:
num = 100

plt.subplot(121)
plt.imshow(color_X[num], 'gray')

plt.subplot(122)
plt.imshow(gray_X[num], 'gray')

plt.show()

In [ ]:
color_X = (color_X/255.0).reshape(-1, 32*32*3)
gray_X = (gray_X/255.0).reshape(-1, 32, 32, 1)

In [ ]:
import tensorflow as tf

learning_rate = 0.1
epochs = 100
train_step = 0

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, 32, 32, 1))
Y = tf.placeholder(tf.float32, shape=(None, 32*32*3))

def conv2d(layer, name, filters, stddev):
    activation_fn = tf.nn.relu
    return tf.layers.conv2d(layer, name=name, filters=filters,\
                          activation=activation_fn,\
                          kernel_size=(3, 3), strides=(1,1),\
                          padding='SAME',\
                          kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=1e-2, dtype=tf.float32),\
                          bias_initializer=tf.zeros_initializer(),\
                          trainable=True, use_bias=True)

def fc(layer, name, size, stddev):
    activation_fn = tf.nn.relu
    return tf.layers.dense(layer, size, activation=activation_fn,\
                                name=name, trainable=True,\
                                kernel_initializer=tf.truncated_normal_initializer(mean=0.0, dtype=tf.float32, stddev=stddev),\
                                bias_initializer=tf.zeros_initializer(),\
                                kernel_regularizer=tf.nn.l2_loss,\
                                use_bias=True)

conv_1 = conv2d(X,"conv_1", 64, 1e-2)
max_pool_1 = tf.layers.max_pooling2d(conv_1, (2, 2), (2, 2), padding='SAME')

conv_2 = conv2d(max_pool_1,"conv_2", 128, 1e-2)
max_pool_2 = tf.layers.max_pooling2d(conv_2, (2, 2), (2, 2), padding='SAME')

conv_3 = conv2d(max_pool_2,"conv_3", 256, 1e-2)

conv_out = tf.reshape(conv_3, shape=(-1, 8*8*256))


fc_1 = fc(conv_out, 'fc_1', 4096, 1e-2)

logits = fc(fc_1, 'logits', 32*32*3, 1e-2)

loss = tf.losses.mean_squared_error(labels=tf.reshape(Y, shape=(-1, 32*32*3)), predictions=logits)

train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in xrange(100):
        val, _ = sess.run([loss, train_op], feed_dict={X: gray_X[:200], Y: color_X[:200]})
        print "Loss: ", val
